In [74]:
import numpy as np
import pandas as pd
import json
import gc
from collections import defaultdict
import sys
from enum import Enum
import random
import pickle

random.seed(824)

print("Setup complete!")

Setup complete!


In [75]:
a_file = open("prepared_data/data-small.pkl", "rb")
data = pickle.load(a_file)
a_file.close()
movies_feat = data["movies_feat"]
movie_map = data["movie_map"]
movie_adj_list = data["movie_adj_list_train"]
user_adj_list = data["user_adj_list_train"]
edge_list_train = data["edge_list_train"]
edge_list_valid = data["edge_list_valid"]
edge_list_test = data["edge_list_test"]


In [76]:
pos_to_idx = dict((y,x) for x, y in movie_map.items())

In [77]:
dataset_dir = "../datasets/ml-25m"
movies_df = pd.read_csv(dataset_dir + "/movies.csv")
movies_df = movies_df.set_index("movieId")
movies_df

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
209157,We (2018),Drama
209159,Window of the Soul (2001),Documentary
209163,Bad Poems (2018),Comedy|Drama


In [78]:
pos = 2301
index = pos_to_idx[pos]
movies_df.loc[index]

title     For Your Eyes Only (1981)
genres    Action|Adventure|Thriller
Name: 2989, dtype: object

In [79]:
a_file = open("../models/20210808-231009/recommends.pkl", "rb")
personal_ratings = pickle.load(a_file)
a_file.close()

In [80]:
print(len(personal_ratings))
print(personal_ratings[0])

8726
3.5414512157440186


In [81]:
movie_ratings = dict()
for i in range(len(movie_adj_list)):
    num = len(movie_adj_list[i])
    sum = 0
    for rate in movie_adj_list[i]:
        sum += rate[1]
    movie_ratings[i] = sum / num
print(len(movie_ratings))
print(movie_ratings[0])

8726
3.8731404958677684


In [82]:
razlike = dict()
for i in range(len(movie_ratings)):
    razlike[i] = personal_ratings[i] - movie_ratings[i] 
print(len(razlike))
print(razlike[0])

8726
-0.3316892801237499


In [48]:
print(max(razlike.values()))

3.1939712524414063


In [51]:
naj = 0
for i in range(len(razlike)):
    if razlike[i] > naj:
        naj = razlike[i]
print(naj)

3.1939712524414063


In [83]:
max_razlika = max(razlike.values())
max_personal_rating = max(personal_ratings.values())
print(max_razlika)
print(max_personal_rating)
for i in range(len(personal_ratings)):
    personal_ratings[i] /= max_personal_rating
for i in range(len(razlike)):
    razlike[i] /= max_razlika

print(razlike[0])
print(personal_ratings[0])

2.8124493492974176
5.464280605316162
-0.11793609019362773
0.6481093251870272


In [84]:
best = dict()
for i in range(len(razlike)):
    best[i] = razlike[i] + personal_ratings[i]
print(len(best))
print(best[0])

8726
0.5301732349933995


In [85]:
to_watch = sorted(best.items(), key=lambda x: x[1], reverse=True)[:10]

In [86]:
print(to_watch)

[(4401, 1.657035644103322), (4450, 1.6062651258957217), (6098, 1.5630900692571807), (5629, 1.548475395052495), (6112, 1.5206424804350545), (1930, 1.5007737607312799), (8131, 1.4766063708457513), (8645, 1.4758427893489703), (6170, 1.4715532391559916), (5590, 1.436012969433299)]


In [92]:
pos = 6112
index = pos_to_idx[pos]
movies_df.loc[index]

title     Seven Up! (1964)
genres         Documentary
Name: 50740, dtype: object

In [60]:
good_users = []
for i in range(len(user_adj_list)):
    if len(user_adj_list[i]) <= 12:
        good_users.append(i)

print(len(good_users))

75


In [61]:
print(good_users)

[69, 149, 344, 582, 636, 680, 748, 788, 821, 908, 1052, 1083, 1085, 1121, 1152, 1193, 1281, 1292, 1310, 1400, 1469, 1713, 1797, 1902, 1975, 2040, 2057, 2176, 2198, 2230, 2251, 2261, 2351, 2364, 2398, 2423, 2443, 2510, 2678, 2802, 2823, 2852, 2900, 2902, 2934, 3019, 3076, 3124, 3157, 3186, 3221, 3347, 3358, 3395, 3542, 3654, 3658, 3684, 3694, 3697, 3761, 3783, 3806, 3837, 3852, 3938, 4008, 4081, 4082, 4170, 4255, 4351, 4434, 4649, 4793]
